In [ ]:
import pandas as pd
import numpy as np
from sklearn.metrics.pairwise import cosine_similarity

data = pd.read_excel('/content/drive/MyDrive/캡스톤/캡스톤_당뇨식단.xlsx')

In [ ]:
data.head()

,하루칼로리,메뉴구분,메뉴1,메뉴2,메뉴3,메뉴4,메뉴5,메뉴6,육류,어류,면류,밥류,매운맛
0,1400,아침,오곡밥 2/3공기(밥 140g),우거지된장국(우거지 50g),불고기(쇠고기 40g),"오이달래무침(오이 40g, 달래 10g)",숙주나물(숙주 70g),나박김치,1.0,NaN,NaN,1.0,NaN
1,1400,점심,현미밥 1공기(밥 210g),콩나물국(콩나물 40g),조기구이(조기 50g),파프리카샐러드(파프리카 40g),고사리나물(고사리 40g),배추김치,NaN,1.0,NaN,1.0,NaN
2,1400,간식,토마토(중) 2개,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,1400,저녁,잡곡밥 2/3공기(밥 140g),두부찌개(두부 80g),닭채소조림(닭고기 40g),시금치나물(시금치 70g),무생채(무 50g),열무김치,1.0,NaN,NaN,1.0,NaN
4,1400,간식,우유 1컵(200ml),NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [ ]:
data=data.fillna(0)

In [ ]:
# 사용자 입력 받기
preference_meat = int(input("육류 선호도(0 또는 1): "))
preference_fish = int(input("어류 선호도(0 또는 1): "))
preference_noodles = int(input("면류 선호도(0 또는 1): "))
preference_rice = int(input("밥류 선호도(0 또는 1): "))
preference_spicy = int(input("매운맛 선호도(0 또는 1): "))
gender = input("성별(남성 또는 여성): ")
height = float(input("키(cm): "))
weight = float(input("몸무게(kg): "))
activity_level = input("활동량(가벼운 활동, 중증도 활동, 심한 활동): ")

# 표준체중 및 1일 총 열량 계산
if gender == '남성':
    standard_weight = (height / 100) ** 2 * 22
else:
    standard_weight = (height / 100) ** 2 * 21

if activity_level == '가벼운 활동':
    calorie_range = (standard_weight * 25, standard_weight * 30)
elif activity_level == '중증도 활동':
    calorie_range = (standard_weight * 30, standard_weight * 35)
elif activity_level == '심한 활동':
    calorie_range = (standard_weight * 35, standard_weight * 40)
else:
    print("잘못된 활동량을 입력하셨습니다.")
    exit()

# 필요한 열량 계산
target_calories = (calorie_range[0] + calorie_range[1]) / 2  # 필요한 열량 범위의 중간값으로 설정

# 필요한 열만 선택
filtered_data = data[['하루칼로리', '메뉴구분', '메뉴1', '메뉴2', '메뉴3', '메뉴4', '메뉴5', '메뉴6', '육류', '어류', '면류', '밥류', '매운맛']]

# 추천 식단 생성
if target_calories <= 1500:
    recommended_calories = 1400
elif target_calories <= 1700:
    recommended_calories = 1600
elif target_calories <= 1900:
    recommended_calories = 1800
else:
    recommended_calories = 2000

# 아침, 점심, 저녁 식단 추천
filtered_data_meals = filtered_data[
    (filtered_data['메뉴구분'] == '아침') |
    (filtered_data['메뉴구분'] == '점심') |
    (filtered_data['메뉴구분'] == '저녁')
]

filtered_data_meals = filtered_data_meals[
    (filtered_data_meals['하루칼로리'] == recommended_calories) &
    (
        (
            (preference_meat == 1) & ((filtered_data_meals['육류'] == 1) | (filtered_data_meals['육류'] == 0)) |
            (preference_fish == 1) & ((filtered_data_meals['어류'] == 1) | (filtered_data_meals['어류'] == 0)) |
            (preference_noodles == 1) & ((filtered_data_meals['면류'] == 1) | (filtered_data_meals['면류'] == 0)) |
            (preference_rice == 1) & ((filtered_data_meals['밥류'] == 1) | (filtered_data_meals['밥류'] == 0)) |
            (preference_spicy == 1) & ((filtered_data_meals['매운맛'] == 1) | (filtered_data_meals['매운맛'] == 0))
        ) |
        (
            (preference_meat == 0) & (preference_fish == 0) & (preference_noodles == 0) & (preference_rice == 0) & (preference_spicy == 0)
        )
    ) &
    (
        ~(
            (preference_meat == 0) & (filtered_data_meals['육류'] == 1) |
            (preference_fish == 0) & (filtered_data_meals['어류'] == 1) |
            (preference_noodles == 0) & (filtered_data_meals['면류'] == 1) |
            (preference_rice == 0) & (filtered_data_meals['밥류'] == 1) |
            (preference_spicy == 0) & (filtered_data_meals['매운맛'] == 1)
        )
    )
]


# 아침, 점심, 저녁 각각 1개의 식단 랜덤 추천
recommended_meals = []
meal_types = ['아침', '점심', '저녁']
for meal_type in meal_types:
    filtered_meals_type = filtered_data_meals[filtered_data_meals['메뉴구분'] == meal_type]
    if len(filtered_meals_type) > 0:
        random_meal_index = np.random.randint(0, len(filtered_meals_type))
        recommended_meal = filtered_meals_type.iloc[random_meal_index]
        recommended_meals.append(recommended_meal)
    else:
        # 추천되는 식단이 없는 경우 다른 식단 카테고리에서 선택
        other_meal_types = [m for m in meal_types if m != meal_type]
        for other_meal_type in other_meal_types:
            filtered_meals_type = filtered_data_meals[filtered_data_meals['메뉴구분'] == other_meal_type]
            if len(filtered_meals_type) > 0:
                random_meal_index = np.random.randint(0, len(filtered_meals_type))
                recommended_meal = filtered_meals_type.iloc[random_meal_index]
                recommended_meals.append(recommended_meal)
                break

# 간식 식단 추천
filtered_data_snacks = filtered_data[filtered_data['메뉴구분'] == '간식']
if len(filtered_data_snacks) > 0:
    random_snack_index1 = np.random.randint(0, len(filtered_data_snacks))
    recommended_snack1 = filtered_data_snacks.iloc[random_snack_index1]
    recommended_meals.insert(2, recommended_snack1)

    # 다른 간식 식단 추천
    filtered_data_snacks2 = filtered_data_snacks[filtered_data_snacks.index != random_snack_index1]
    if len(filtered_data_snacks2) > 0:
        random_snack_index2 = np.random.randint(0, len(filtered_data_snacks2))
        recommended_snack2 = filtered_data_snacks2.iloc[random_snack_index2]
        recommended_meals.append(recommended_snack2)
    else:
        recommended_meals.append(recommended_snack1)
        print("다른 간식 식단이 없어 같은 간식 식단이 중복으로 추가되었습니다.")
else:
    print("추천할 간식이 없습니다.")

# 최종 추천 식단 출력
print("=== 추천 식단 ===")
meal_types = ['아침', '점심', '간식', '저녁', '간식']
for meal_type, meal in zip(meal_types, recommended_meals):
    menu = [meal[f'메뉴{i+1}'] for i in range(6)]
    menu_str = ', '.join([m if str(m) != '0' else '' for m in menu])
    print(f"{meal_type}: {menu_str.replace(', ,', '')}")

육류 선호도(0 또는 1): 1
어류 선호도(0 또는 1): 0
면류 선호도(0 또는 1): 1
밥류 선호도(0 또는 1): 1
매운맛 선호도(0 또는 1): 0
성별(남성 또는 여성): 남성
키(cm): 180
몸무게(kg): 77
활동량(가벼운 활동, 중증도 활동, 심한 활동): 중증도 활동
=== 추천 식단 ===
아침: 오곡밥 2/3공기(밥 140g), 근대된장국, 달걀찜(달걀(중) 1개), 돼지고기부추볶음(돼지고기(안심) 40g), 쑥갓나물, 배추김치
점심: 콩국수(국수(삶) 270g, 1.5공기, 콩국물 1대접), 오이나물, 부추무침, 배추김치 
간식: 두유 1컵(200ml)  , 
저녁: 현미밥 1공기(밥 210g), 콩비지찌개(콩비지 180g), 수육겨자채(수육 40g), 곤약숙회, 깻잎무침, 배추김치
간식: 우유 1컵(200ml)  , 
